In [1]:
import numpy as np

In [2]:
from Expert_Prediction.online_experts import *
from Ensemble_Methods.ensemble_algorithms import *
from ensemble import *

# Building Forecasters

In [3]:
"Training Models"
actual_values = np.random.randint(0,10,size=[40])
def create_expert(std,actual_values):
    return actual_values+np.random.randn(actual_values.size)*std

# Fixed Standard Deviations for each expert
stds = [1,0.1,0.03,0.5,1.3,2,5,1,2,5]
expert_predictions = np.array([create_expert(std,actual_values) for std in stds])

In [4]:
experts = [CannedExpert(expert_predictions[i]) for i in range(10)]

In [5]:
experts[0]._predict(1)

8.919599630547486

# Building Ensemble Forecaster

In [6]:
ensemble_forecaster = EnsembleForecaster(experts,NNLSEnsemble(10,loss_func=se))

In [9]:
prediction = ensemble_forecaster._predict(1)

In [10]:
prediction

Prescription for training:
- 

In [ ]:
def calculate_losses(predicted,actual,loss_func = None):
    assert len(predicted)==len(actual)
    total_length = len(predicted)
    
    output = [loss_func(predicted[i],actual[i]) for i in range(total_length)]
    return output

In [ ]:
predictions = []
for i in range(21,40):
    predictions.append(test5._predict(expert_predictions[:,i]))
    test5._update(expert_predictions[:,i][:,np.newaxis],actual_values[i,np.newaxis])
plt.plot(predictions,label="Predictions")
plt.plot(actual_values[21:40],label="Actual Values")
plt.legend()

We can either
- train the ensemble while it is running
- train the ensemble before it is running
- train the ensemble while it is running and then also pre-train it
- cold start it

Pre-train requires online

In [ ]:
 how many past values to include, i.e. the
size of the embed

In [ ]:
def _fit_expert(X)

Would it make sense to pretrain the models to begin with. YES! Forecast with SKTIME and the other one.

In [1]:
class ForecastingExpert(Expert):
    def __init__(self,outputs):
    
    def _predict(self):
    
    def _fit(self,train_data):

IndentationError: expected an indented block (<ipython-input-1-116e96030850>, line 4)

In [2]:
from sktime.forecasters.base import BaseForecaster
from sktime.forecasters.base import BaseSingleSeriesForecaster
from sktime.forecasters.base import BaseUpdateableForecaster

In [ ]:
class EnsembleForecaster(BaseForecaster):
    """
    Ensemble of forecasters.
    Parameters
    ----------
    estimators : list of (str, estimator) tuples
        List of (name, estimator) tuples.
    weights : array-like, shape = [n_estimators], optional (default=None)
        Sequence of weights (float or int) to weight the occurrences of predicted values before averaging.
        Uses uniform weights if None.
    check_input : bool, optional (default=True)
        - If True, input are checked.
        - If False, input are not checked and assumed correct. Use with caution.
    """

    # TODO: experimental, major functionality not implemented (input checks, params interface, exogenous variables)

    def __init__(self, estimators=None, weights=None, check_input=True):
        # TODO add input checks
        self.estimators = estimators
        self.weights = weights
        self.fitted_estimators_ = []
        super(EnsembleForecaster, self).__init__(check_input=check_input)

    def fit_estimators(self, y, fh=1, X=None):
        """
        Internal fit.
        Parameters
        ----------
        y : pandas.Series
            Target time series to which to fit the forecaster.
        X : pandas.DataFrame, shape=[n_obs, n_vars], optional (default=None)
            An optional 2-d dataframe of exogenous variables. If provided, these
            variables are used as additional features in the regression
            operation. This should not include a constant or trend. Note that
            if an ``ARIMA`` is fit on exogenous features, it must also be provided
            exogenous features for making predictions.
        Returns
        -------
        self : returns an instance of self.
        """
        # validate forecasting horizon
        fh = validate_fh(fh)

        # Clear previously fitted estimators
        self.fitted_estimators_ = []

        for _, estimator in self.estimators:
            # TODO implement set/get params interface
            # estimator.set_params(**{"check_input": False})
            fitted_estimator = estimator.fit(y, fh=fh, X=X)
            self.fitted_estimators_.append(fitted_estimator)
        return self

    def predict(self, fh=1, X=None):
        """
        Internal predict using fitted estimator.
        Parameters
        ----------
        fh : array-like, optional (default=None)
            The forecasters horizon with the steps ahead to to predict. Default is one-step ahead forecast,
            i.e. np.array([1])
        X : pandas.DataFrame, shape=[n_obs, n_vars], optional (default=None)
            An optional 2-d dataframe of exogenous variables. If provided, these
            variables are used as additional features in the regression
            operation. This should not include a constant or trend. Note that if
            provided, the forecaster must also have been fitted on the exogenous
            features.
        Returns
        -------
        Predictions : pandas.Series, shape=(len(fh),)
            Returns series of predicted values.
        """
        # TODO pass X only to estimators where the predict method accepts X, currenlty X is ignored

        # Forecast all periods from start to end of pred horizon, but only return given time points in pred horizon
        fh = validate_fh(fh)
        fh_idx = fh - np.min(fh)

        # Iterate over estimators
        y_preds = np.zeros((len(self.fitted_estimators_), len(fh)))
        indexes = []
        for i, estimator in enumerate(self.fitted_estimators_):
            y_pred = estimator.predict(fh=fh)
            y_preds[i, :] = y_pred
            indexes.append(y_pred.index)

        # Check if all predicted horizons are identical
        if not all(index.equals(indexes[0]) for index in indexes):
            raise ValueError('Predicted horizons from estimators do not match')

        # Average predictions over estimators
        avg_preds = np.average(y_preds, axis=0, weights=self.weights)

        # Return average predictions with index
        index = indexes[0]
        name = y_preds[0].name if hasattr(y_preds[0], 'name') else None
        return pd.Series(avg_preds, index=index, name=name)

    def get_params(self, deep=True):
        # TODO fix get and set params interface following sklearn double underscore convention
        raise NotImplementedError()

    def set_params(self, **params):
        raise NotImplementedError()